# Style-Transfer with Pytorch
Code taken from [NEURAL TRANSFER USING PYTORCH](https://pytorch.org/tutorials/advanced/neural_style_tutorial.html?highlight=style%20transfer).

## 1. Setup:

### 1.1. Import libs

In [11]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms

import torchvision.models as models

import copy

import numpy as np

### 1.2. Choose machine

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1.3. Define the desired size of the output image
Don't go too high 'cause this may stress your machine. Start small and work your way up, benchmarking your machine at the same time.

In [16]:
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

## 2. Load Image

### 2.1. Import and Rescale image
The `loader` simply imports the provided image and does a bit of resizing based on the `imsize` value. Then converts the image into a tensor. 

In [18]:
loader = transforms.Compose([
    transforms.Resize(imsize), # scale imported image
    transforms.ToTensor()])    # transform it into a torch tensor

We use a helper function to actually apply the loader onto the images we want to process:

In [19]:
def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [20]:
content_img = image_loader("./data/images/neural-style/dancing.jpg")
style_img = image_loader("./data/images/neural-style/picasso.jpg")


FileNotFoundError: [Errno 2] No such file or directory: './data/images/neural-style/picasso.jpg'

In [21]:
assert style_img.size() == content_img.size(), \
    "we need to import style and content images of the same size"

NameError: name 'style_img' is not defined